In [9]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense


In [31]:
# read clean electricity data to create target variable array y
elec = pd.read_csv('../data/elec_clean.csv',index_col=0)
elec.index = pd.to_datetime(elec.index,format='%Y-%m-%d %H:00:00')

# remove Davenport for now because it is missing most of June and July
elec.drop('Davenport',axis=1,inplace=True)

agg = pd.Series(elec.sum(axis=1))

In [133]:
num_days = 20
start = '2018-02-01 09:00:00'
split_times = pd.date_range(start=start,freq='D',periods=num_days)
train_list, test_list = list(), list()

for k in range(num_days):
    train_entry = np.array(agg[split_times[k]-pd.Timedelta(hours=168):split_times[k]-pd.Timedelta(hours=1)])
    test_entry = np.array(agg[split_times[k]:split_times[k]+pd.Timedelta(hours=38)])
    train_list.append(train_entry)
    test_list.append(test_entry)

train = np.array(train_list).reshape(num_days,168,1)
test = np.array(test_list).reshape(num_days,39,1)


(20, 168, 1)

In [ ]:
# define model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(168, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(train, y, epochs=200, verbose=0)